# libraries and imports

library to scrape sec edgar website
https://github.com/coyo8/sec-edgar

In [1]:
import pandas as pd
#import regex as re
import datetime
#import os 
import numpy as np

In [2]:
# import libraries and modules
from sklearn.metrics import accuracy_score, confusion_matrix
from sklearn.model_selection import train_test_split
import seaborn as sns
from matplotlib import pyplot as plt
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer, HashingVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score, GridSearchCV 
from sklearn.neighbors import KNeighborsClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import BaggingClassifier
from sklearn.ensemble import RandomForestClassifier, ExtraTreesClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn import svm
from sklearn.svm import SVC
from sklearn.naive_bayes import BernoulliNB
def warn(*args, **kwargs):
    pass
import warnings
warnings.warn = warn

## Import data

In [3]:
# cluster
#['aapl' 'amzn' 'csco' 'ebay' 'fcbk' 'msft']

In [4]:
aapl=pd.read_csv('../csv/aapl_8.csv', index_col=0)
#goog=pd.read_csv('../csv/goog_8.csv', index_col=0)
msft=pd.read_csv('../csv/msft_8.csv', index_col=0)
#adbe=pd.read_csv('../csv/adbe_8.csv', index_col=0)
#akam=pd.read_csv('../csv/akam_8.csv', index_col=0)
fcbk=pd.read_csv('../csv/fcbk_8.csv', index_col=0)
amzn=pd.read_csv('../csv/amzn_8.csv', index_col=0)
#ibm=pd.read_csv('../csv/ibm_8.csv', index_col=0)
#intc=pd.read_csv('../csv/intc_8.csv', index_col=0)
csco=pd.read_csv('../csv/csco_8.csv', index_col=0)
#ea=pd.read_csv('../csv/ea_8.csv', index_col=0)
ebay=pd.read_csv('../csv/ebay_8.csv', index_col=0)

In [5]:
comps = aapl
#comps = aapl.append(goog, ignore_index=True, sort=None)
comps = comps.append(msft, ignore_index=True, sort=None)
#comps = comps.append(adbe, ignore_index=True, sort=None)
#comps = comps.append(akam, ignore_index=True, sort=None)
comps = comps.append(fcbk, ignore_index=True, sort=None)
comps = comps.append(amzn, ignore_index=True, sort=None)
#comps = comps.append(ibm, ignore_index=True, sort=None)
#comps = comps.append(intc, ignore_index=True, sort=None)
comps = comps.append(csco, ignore_index=True, sort=None)
#comps = comps.append(ea, ignore_index=True, sort=None)
comps = comps.append(ebay, ignore_index=True, sort=None)
comps.to_csv('../csv/stocks.csv')

In [7]:
# sort by date so that companies are shuffled (test not based on few companies not in train)
comps.sort_values(by=['date'],inplace=True)
comps.head(1)

,date,ticker,neg_text,uncert_text,pos_text,delta_0_p,delta_1_p,delta_1_0,delta_3_np,delta_7_np,delta_3_n0,delta_7_n0,period,an_buy,an_hold,an_sell,an_rating,an_d_p,an_d_0
253,2009-04-17,amzn,against break challenge challenges closure con...,appear believe could differ may might pending ...,able achieving advantage advantages benefit be...,1,1,0,0,0,1,0,6.0,9.0,20.0,3.0,3.406,1,1


In [8]:
comps.reset_index(inplace=True)

#### Check Data

In [11]:
comps.ticker.unique()

array(['amzn', 'ebay', 'aapl', 'msft', 'fcbk', 'csco'], dtype=object)

## Prepare data for right format

In [12]:
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder
tickers = comps.ticker
ticker_encoder = LabelEncoder()
ticker_encoded = ticker_encoder.fit_transform(tickers)

In [13]:
comps['ticker'] = ticker_encoded

## Model

### Features and Target

In [16]:
# set text feature and target
X = comps['neg_text'] #alternatively: pos_text uncert_text
y = comps['delta_7_np']
#y = pd.DataFrame(y)

### Train-test-split

In [17]:
comps.shape

(524, 20)

In [18]:
# Create train_test_split.
X_train, X_test, y_train, y_test = train_test_split(X, y, 
                                                    test_size = 0.2,
                                                    random_state = 42,
                                                    shuffle=False
                                                   )

In [20]:
X_train.shape, y_train.shape, X_test.shape, y_test.shape, 

((419,), (419,), (105,), (105,))

### Classifers and Vectorizers

In [21]:
# vectorizer initialized (also see GS 506)
cvec = CountVectorizer()
tf = TfidfVectorizer()
svc = svm.SVC()
nb = BernoulliNB()

In [46]:
# countVectorizer and Multinomial Naive Bayes are instantiated

pipe = Pipeline([
    ('vec', tf),
    ('clf', svc),
    #('ss', StandardScaler())
    ])

In [52]:
# best of TFIDF: 
pipe_params = {
    'vec__max_features' : [400],
    'clf__C': [7],
    'clf__gamma':[0.003],
    'vec__use_idf' : [True],
    'vec__norm':['l2'],
    'vec__min_df' : [0],
    'vec__ngram_range':[(1,1)],
}
gs = GridSearchCV(pipe, param_grid= pipe_params, cv=3)
gs.fit(X_train, y_train)
gs.best_score_, gs.best_params_

(0.5727923627684964,
 {'clf__C': 7,
  'clf__gamma': 0.003,
  'vec__max_features': 400,
  'vec__min_df': 0,
  'vec__ngram_range': (1, 1),
  'vec__norm': 'l2',
  'vec__use_idf': True})

In [51]:
#best of CVEC: 
pipe_params = {
    'vec__max_features' : [520],
    'clf__C': [7],
    'clf__gamma':[0.003],
    'vec__min_df' : [3],
    'vec__max_df':[100],
    'vec__ngram_range':[(1,3)],
    'vec__binary':[True]
}
gs = GridSearchCV(pipe, param_grid= pipe_params, cv=5)
gs.fit(X_train, y_train)
gs.best_score_, gs.best_params_

(0.5727923627684964,
 {'clf__C': 7,
  'clf__gamma': 0.003,
  'vec__binary': True,
  'vec__max_df': 100,
  'vec__max_features': 520,
  'vec__min_df': 3,
  'vec__ngram_range': (1, 3)})

### Baseline prediction

In [53]:
round(max(comps['delta_7_np'].value_counts(normalize=True)*100),2)

58.97